# Göksu Güz 150180715

# The video named 'hw4_part1_missing' is also the result of one of my trials, but in a way that I don't understand how i lost the code of it, however video remained, so i had to rewrite part 1 :( 

# That version was better so i wanted to send that video too.

In [103]:
import numpy as np
import cv2 
from scipy import signal
from matplotlib import pyplot as plt
import os
from matplotlib.patches import Circle

In [104]:
directory = "DJI_0101/"
count = 0
window_size = 5;
w = window_size//2
images_ = []

In [105]:
for fileName in os.listdir(directory):
    imageFrom_ = directory + fileName
    image = cv2.imread(imageFrom_)
    new_corners = []
    
    if(count == 0):
        beforeImage = image
        
    if(count >= 1 and count%5 == 0):
        #Gaussian 
        image1 = cv2.GaussianBlur(image, (15,15), 1)
        beforeImage1 = cv2.GaussianBlur(beforeImage, (15,15), 1)

        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        beforeImage1 = cv2.cvtColor(beforeImage1, cv2.COLOR_BGR2GRAY)
        image_new = image.copy()
        
        #print(image1.shape)
        corners = cv2.goodFeaturesToTrack(image1,5,0.4,100) 
        #print(corners.shape)
        x_points = corners[:,0,0]
        y_points = corners[:,0,1]
        
        for i in range(y_points.shape[0]):
            point = (int(x_points[i]),int(y_points[i])) 
            new_corners.append(point)
        #real_points = new_corners
        
        image1 = image1/255.
        beforeImage1 = beforeImage1/255.
        
        new_corners = np.array(new_corners)
        
        for i in new_corners:
            cv2.circle(image_new, i, 3, (255, 0, 0), -1)  
        #cv2.imshow("img", image_new)
        #cv2.waitKey(0) 
        
        new_img_1 = image.copy()
        
        for l in new_corners:
            j,i = l.flatten()

            mode = "valid"
            temp = 1
            Ix = signal.convolve2d(beforeImage1,[[-temp,temp],[-temp,temp]],mode = mode) 
            Iy = signal.convolve2d(beforeImage1,[[-temp,-temp],[temp,temp]],mode = mode) 
            It = signal.convolve2d(beforeImage1,[[temp,temp],[temp,temp]],mode = mode) 
            + signal.convolve2d(image1,[[-temp,-temp],[-temp,-temp]],mode = mode)

            IX = Ix[i-w:i+w+1, j-w:j+w+1]
            IY = Iy[i-w:i+w+1, j-w:j+w+1]
            IT = It[i-w:i+w+1, j-w:j+w+1]

            A_transposeA = np.array([[np.sum(IX**2), np.sum(IX * IY)], [np.sum(IY * IX), np.sum(IY**2)]])
            A_b = -np.array([np.sum(IX * IT), np.sum(IY*IT)])
            inversedA = np.linalg.pinv(A_transposeA)
            u,v = np.dot(inversedA, A_b)
        
            first_point = (j, i)
            #print(first_point)
            second_point = int(j + int(u*2)),int(i + int(v*2))
            
            if(abs(u)>0.2 or abs(v)>0.2):           
                cv2.arrowedLine(new_img_1, first_point, second_point ,(0, 255, 0), 1,tipLength = 0.3)
                
        #cv2.imshow("img", new_img_1)
        #cv2.waitKey(0)   
        images_.append(new_img_1)
    beforeImage = image
    count += 1 
    
    

In [106]:
import moviepy.editor as mpy
clip = mpy.ImageSequenceClip(images_, fps = 10)
clip.write_videofile("hw4_part1.mp4", codec="libx264")

t:  29%|███████████████████▍                                                | 26/91 [00:00<00:00, 250.78it/s, now=None]

Moviepy - Building video hw4_part1.mp4.
Moviepy - Writing video hw4_part1.mp4




t:  79%|█████████████████████████████████████████████████████▊              | 72/91 [21:34<00:00, 269.69it/s, now=None]

Moviepy - Done !
Moviepy - video ready hw4_part1.mp4
